In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler
from datasets import load_dataset
import tqdm
from transformers import GPT2TokenizerFast, BertTokenizerFast, Trainer, TrainingArguments, EvalPrediction
import re
import tiktoken
import numpy as np
from bus_nGPT import Decoder, TransformerLayer, AttentionHead, Rotary, LMHead
import tensorboard as tb
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from importlib import reload

2024-12-04 13:24:39.931684: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733340279.947741 1549815 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733340279.952930 1549815 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-04 13:24:39.970463: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
baseline = torch.load('baseline_model.pt')

/tmp/ipykernel_1549815/1169377317.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  baseline = torch.load('baseline_model.pt')


In [3]:
snli = load_dataset('stanfordnlp/snli')
mnli = load_dataset('nyu-mll/multi_nli')

In [4]:
tokenizer = BertTokenizerFast.from_pretrained('google-bert/bert-base-uncased')

In [5]:
def tokenize_fn(examples):
    return tokenizer(examples['premise'] + " " + examples['hypothesis'], 
                     padding='max_length', 
                     truncation=True,
                     )

In [6]:
def compute_accuracy(eval: EvalPrediction):
    x = np.argmax(eval.predictions[0], axis=-1)
    z = np.average(x == eval.predictions[1])
    return {
        'accuracy': z    
        }

In [7]:
def abs_func(example):
    return {'label': example['label']} if example['label'] >= 0 else {'label':-example['label']}

In [8]:
snli = snli.map(abs_func)

In [9]:
# snli_tokenized = snli.map(tokenize_fn)
mnli_tokenized = mnli.map(tokenize_fn)

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

In [20]:
bl = LMHead(baseline, 512, 3)
bl.to('cuda')

LMHead(
  (ffn): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): GELU(approximate='none')
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=512, out_features=3, bias=True)
  )
  (model): Decoder(
    (blocks): ModuleList(
      (0-7): 8 x TransformerLayer(
        (heads): ModuleList(
          (0-7): 8 x AttentionHead(
            (rope): Rotary()
          )
        )
        (silu): SiLU()
      )
    )
    (embeddings): Embedding(30523, 512)
  )
)

In [21]:

training_args = TrainingArguments(
	output_dir="output", 
	eval_strategy="steps", 
	num_train_epochs = 2,
	warmup_steps = 0,
	logging_steps = 100,
	save_steps = 100,
	load_best_model_at_end = True,
	learning_rate = 1e-3,
    per_device_train_batch_size=8,
    label_names=['labels'],
    gradient_accumulation_steps=195,
	)

trainer = Trainer(
    model=bl,
    args=training_args,
    train_dataset=mnli_tokenized['train'],
    eval_dataset=mnli_tokenized['validation_mismatched'],
	tokenizer=tokenizer,
    compute_metrics=compute_accuracy

)



/tmp/ipykernel_1549815/1813546762.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
trainer.train()

  0%|          | 0/502 [00:00<?, ?it/s]

{'loss': 1.099, 'grad_norm': 20.157278060913086, 'learning_rate': 0.0008007968127490039, 'epoch': 0.4}


  0%|          | 0/1229 [00:00<?, ?it/s]

{'eval_loss': 1.0968862771987915, 'eval_accuracy': 0.34814890154597233, 'eval_runtime': 56.5619, 'eval_samples_per_second': 173.827, 'eval_steps_per_second': 21.728, 'epoch': 0.4}
{'loss': 1.0977, 'grad_norm': 7.472654342651367, 'learning_rate': 0.000601593625498008, 'epoch': 0.79}


  0%|          | 0/1229 [00:00<?, ?it/s]

...

  0%|          | 0/1229 [00:00<?, ?it/s]

{'eval_loss': 1.091715693473816, 'eval_accuracy': 0.37794955248169243, 'eval_runtime': 57.3025, 'eval_samples_per_second': 171.581, 'eval_steps_per_second': 21.448, 'epoch': 1.59}
{'loss': 1.0927, 'grad_norm': 4.646554470062256, 'learning_rate': 3.98406374501992e-06, 'epoch': 1.99}


  0%|          | 0/1229 [00:00<?, ?it/s]

{'eval_loss': 1.0914418697357178, 'eval_accuracy': 0.3708299430431245, 'eval_runtime': 56.4567, 'eval_samples_per_second': 174.151, 'eval_steps_per_second': 21.769, 'epoch': 1.99}
{'train_runtime': 4657.8472, 'train_samples_per_second': 168.62, 'train_steps_per_second': 0.108, 'train_loss': 1.0959256369754138, 'epoch': 1.99}


TrainOutput(global_step=502, training_loss=1.0959256369754138, metrics={'train_runtime': 4657.8472, 'train_samples_per_second': 168.62, 'train_steps_per_second': 0.108, 'total_flos': 0.0, 'train_loss': 1.0959256369754138, 'epoch': 1.9941737288135593})

In [23]:
trainer.evaluate(mnli_tokenized['validation_matched'])

  0%|          | 0/1227 [00:00<?, ?it/s]

{'eval_loss': 1.0917551517486572,
 'eval_accuracy': 0.375649516046867,
 'eval_runtime': 56.6136,
 'eval_samples_per_second': 173.368,
 'eval_steps_per_second': 21.673,
 'epoch': 1.9941737288135593}

In [ ]:
trainer.evaluate(snli_tokenized['test'])

  0%|          | 0/1250 [00:00<?, ?it/s]

{'eval_loss': 1.079468011856079,
 'eval_accuracy': 0.4098,
 'eval_runtime': 61.0158,
 'eval_samples_per_second': 163.892,
 'eval_steps_per_second': 20.487,
 'epoch': 1.9953321991013393}